In [46]:
import pandas as pd
import numpy as np

df = pd.read_csv("./insurance/insurance3r2.csv")


In [47]:
df.head()

,age,sex,bmi,steps,children,smoker,region,charges,insuranceclaim
0,19,0,27.900,3009,0,1,3,16884.92400,1
1,18,1,33.770,3008,1,0,2,1725.55230,1
2,28,1,33.000,3009,3,0,2,4449.46200,0
3,33,1,22.705,10009,0,0,1,21984.47061,0
4,32,1,28.880,8010,0,0,1,3866.85520,1


In [48]:
categ_num = 5

df['age'] = pd.qcut(df['age'],q=10)
df['bmi'] = pd.qcut(df['bmi'],q=10)
df['steps'] = pd.qcut(df['steps'],q=10)
df['charges'] = pd.qcut(df['charges'],q=categ_num)

In [49]:
df.head()

,age,sex,bmi,steps,children,smoker,region,charges,insuranceclaim
0,"(17.999, 19.0]",0,"(27.36, 28.797]","(3007.0, 3010.0]",0,1,3,"(11399.857, 20260.626]",1
1,"(17.999, 19.0]",1,"(33.66, 35.86]","(3007.0, 3010.0]",1,0,2,"(1121.873, 3991.576]",1
2,"(24.0, 29.0]",1,"(32.032, 33.66]","(3007.0, 3010.0]",3,0,2,"(3991.576, 7369.053]",0
3,"(29.0, 34.0]",1,"(15.959000000000001, 22.99]","(8010.0, 10010.0]",0,0,1,"(20260.626, 63770.428]",0
4,"(29.0, 34.0]",1,"(28.797, 30.4]","(8007.0, 8010.0]",0,0,1,"(1121.873, 3991.576]",1


In [50]:
new_df = df.sort_values('charges',inplace=False)
list(new_df['charges'].unique())

[Interval(1121.873, 3991.576, closed='right'),
 Interval(3991.576, 7369.053, closed='right'),
 Interval(7369.053, 11399.857, closed='right'),
 Interval(11399.857, 20260.626, closed='right'),
 Interval(20260.626, 63770.428, closed='right')]

In [52]:
new_df = df.sort_values('bmi',inplace=False)
list(new_df['bmi'].unique())

[Interval(15.959000000000001, 22.99, closed='right'),
 Interval(22.99, 25.326, closed='right'),
 Interval(25.326, 27.36, closed='right'),
 Interval(27.36, 28.797, closed='right'),
 Interval(28.797, 30.4, closed='right'),
 Interval(30.4, 32.032, closed='right'),
 Interval(32.032, 33.66, closed='right'),
 Interval(33.66, 35.86, closed='right'),
 Interval(35.86, 38.619, closed='right'),
 Interval(38.619, 53.13, closed='right')]

In [53]:
def encodeCategory(df,col):
    tempdf = df.sort_values(col,inplace=False)
    valuelist = tempdf[col].unique()
    valuedict = {}
    for i in range(len(valuelist)):
        valuedict[valuelist[i]] = i
    df[col] = df[col].replace(valuedict)
    return df[col]

In [54]:
df["age"] = encodeCategory(df,"age")
df["bmi"] = encodeCategory(df,"bmi")
df["charges"] = encodeCategory(df,"charges")
df["steps"] = encodeCategory(df,"steps")


In [55]:
df

,age,sex,bmi,steps,children,smoker,region,charges,insuranceclaim
0,0,0,3,2,0,1,3,3,1
1,0,1,7,2,1,0,2,0,1
2,2,1,6,2,3,0,2,1,0
3,3,1,0,9,0,0,1,4,0
4,3,1,4,8,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...
1333,7,1,5,5,3,0,1,2,0
1334,0,0,5,0,0,0,0,0,1
1335,0,0,8,2,0,0,2,0,1
1336,1,0,2,8,0,0,3,0,0


In [7]:
X = df.drop(['charges','region','insuranceclaim'], axis = 1)
y = df['charges']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(Dense(6, activation='relu'))
model.add(Dense(36, activation='relu'))
model.add(Dense(categ_num, activation="softmax"))

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics="accuracy")


In [17]:
fitModel =  model.fit(x=X_train,y=y_train,
            validation_data=(X_test,y_test),
            batch_size=1024,epochs=1000,  verbose=1)



Epoch 1/1000
2/2 [==============================] - 0s 75ms/step - loss: 1.6218 - accuracy: 0.1673 - val_loss: 1.6007 - val_accuracy: 0.2537
Epoch 2/1000
2/2 [==============================] - 0s 15ms/step - loss: 1.6115 - accuracy: 0.2262 - val_loss: 1.5909 - val_accuracy: 0.2948
Epoch 3/1000
2/2 [==============================] - 0s 16ms/step - loss: 1.6024 - accuracy: 0.2495 - val_loss: 1.5819 - val_accuracy: 0.3321
Epoch 4/1000
2/2 [==============================] - 0s 14ms/step - loss: 1.5937 - accuracy: 0.2710 - val_loss: 1.5735 - val_accuracy: 0.3433
Epoch 5/1000
2/2 [==============================] - 0s 15ms/step - loss: 1.5853 - accuracy: 0.2897 - val_loss: 1.5652 - val_accuracy: 0.3582
Epoch 6/1000
2/2 [==============================] - 0s 14ms/step - loss: 1.5771 - accuracy: 0.3028 - val_loss: 1.5571 - val_accuracy: 0.3657
Epoch 7/1000
2/2 [==============================] - 0s 15ms/step - loss: 1.5690 - accuracy: 0.3280 - val_loss: 1.5494 - val_accuracy: 0.3694
Epoch 8/1000


Epoch 59/1000
2/2 [==============================] - 0s 14ms/step - loss: 1.2331 - accuracy: 0.5243 - val_loss: 1.1995 - val_accuracy: 0.5448
Epoch 60/1000
2/2 [==============================] - 0s 14ms/step - loss: 1.2267 - accuracy: 0.5290 - val_loss: 1.1931 - val_accuracy: 0.5522
Epoch 61/1000
2/2 [==============================] - 0s 13ms/step - loss: 1.2205 - accuracy: 0.5308 - val_loss: 1.1866 - val_accuracy: 0.5522
Epoch 62/1000
2/2 [==============================] - 0s 13ms/step - loss: 1.2144 - accuracy: 0.5336 - val_loss: 1.1800 - val_accuracy: 0.5522
Epoch 63/1000
2/2 [==============================] - 0s 13ms/step - loss: 1.2082 - accuracy: 0.5364 - val_loss: 1.1730 - val_accuracy: 0.5485
Epoch 64/1000
2/2 [==============================] - 0s 13ms/step - loss: 1.2020 - accuracy: 0.5421 - val_loss: 1.1661 - val_accuracy: 0.5485
Epoch 65/1000
2/2 [==============================] - 0s 14ms/step - loss: 1.1957 - accuracy: 0.5439 - val_loss: 1.1596 - val_accuracy: 0.5485
Epoch 

2/2 [==============================] - 0s 16ms/step - loss: 0.7965 - accuracy: 0.7383 - val_loss: 0.7558 - val_accuracy: 0.7687
Epoch 174/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.7950 - accuracy: 0.7421 - val_loss: 0.7538 - val_accuracy: 0.7687
Epoch 175/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.7934 - accuracy: 0.7411 - val_loss: 0.7519 - val_accuracy: 0.7724
Epoch 176/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.7918 - accuracy: 0.7421 - val_loss: 0.7502 - val_accuracy: 0.7724
Epoch 177/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.7901 - accuracy: 0.7421 - val_loss: 0.7487 - val_accuracy: 0.7761
Epoch 178/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.7884 - accuracy: 0.7467 - val_loss: 0.7472 - val_accuracy: 0.7761
Epoch 179/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.7867 - accuracy: 0.7486 - val_loss: 0.7457 - val_accuracy: 0.7761
Epoch 180/1000

2/2 [==============================] - 0s 15ms/step - loss: 0.6554 - accuracy: 0.7963 - val_loss: 0.6132 - val_accuracy: 0.8358
Epoch 288/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.6542 - accuracy: 0.8000 - val_loss: 0.6123 - val_accuracy: 0.8470
Epoch 289/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.6530 - accuracy: 0.8000 - val_loss: 0.6113 - val_accuracy: 0.8470
Epoch 290/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.6519 - accuracy: 0.8000 - val_loss: 0.6099 - val_accuracy: 0.8470
Epoch 291/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.6510 - accuracy: 0.8009 - val_loss: 0.6088 - val_accuracy: 0.8507
Epoch 292/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.6501 - accuracy: 0.8000 - val_loss: 0.6081 - val_accuracy: 0.8545
Epoch 293/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.6496 - accuracy: 0.8019 - val_loss: 0.6077 - val_accuracy: 0.8470
Epoch 294/1000

2/2 [==============================] - 0s 15ms/step - loss: 0.5738 - accuracy: 0.8196 - val_loss: 0.5383 - val_accuracy: 0.8582
Epoch 402/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.5733 - accuracy: 0.8215 - val_loss: 0.5372 - val_accuracy: 0.8582
Epoch 403/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.5728 - accuracy: 0.8243 - val_loss: 0.5365 - val_accuracy: 0.8545
Epoch 404/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.5723 - accuracy: 0.8262 - val_loss: 0.5357 - val_accuracy: 0.8545
Epoch 405/1000
2/2 [==============================] - 0s 13ms/step - loss: 0.5718 - accuracy: 0.8252 - val_loss: 0.5352 - val_accuracy: 0.8545
Epoch 406/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.5713 - accuracy: 0.8243 - val_loss: 0.5354 - val_accuracy: 0.8545
Epoch 407/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.5709 - accuracy: 0.8262 - val_loss: 0.5354 - val_accuracy: 0.8545
Epoch 408/1000

2/2 [==============================] - 0s 14ms/step - loss: 0.5306 - accuracy: 0.8327 - val_loss: 0.5048 - val_accuracy: 0.8470
Epoch 516/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.5307 - accuracy: 0.8299 - val_loss: 0.5038 - val_accuracy: 0.8470
Epoch 517/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.5309 - accuracy: 0.8280 - val_loss: 0.5028 - val_accuracy: 0.8545
Epoch 518/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.5308 - accuracy: 0.8271 - val_loss: 0.5020 - val_accuracy: 0.8545
Epoch 519/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.5308 - accuracy: 0.8280 - val_loss: 0.5015 - val_accuracy: 0.8582
Epoch 520/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.5309 - accuracy: 0.8271 - val_loss: 0.5015 - val_accuracy: 0.8582
Epoch 521/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.5302 - accuracy: 0.8290 - val_loss: 0.5020 - val_accuracy: 0.8545
Epoch 522/1000

2/2 [==============================] - 0s 15ms/step - loss: 0.5038 - accuracy: 0.8421 - val_loss: 0.4885 - val_accuracy: 0.8545
Epoch 630/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.5040 - accuracy: 0.8402 - val_loss: 0.4892 - val_accuracy: 0.8545
Epoch 631/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.5043 - accuracy: 0.8364 - val_loss: 0.4895 - val_accuracy: 0.8470
Epoch 632/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.5043 - accuracy: 0.8346 - val_loss: 0.4894 - val_accuracy: 0.8545
Epoch 633/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.5041 - accuracy: 0.8402 - val_loss: 0.4889 - val_accuracy: 0.8582
Epoch 634/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.5039 - accuracy: 0.8421 - val_loss: 0.4879 - val_accuracy: 0.8582
Epoch 635/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.5034 - accuracy: 0.8430 - val_loss: 0.4868 - val_accuracy: 0.8619
Epoch 636/1000

2/2 [==============================] - 0s 15ms/step - loss: 0.4918 - accuracy: 0.8458 - val_loss: 0.4710 - val_accuracy: 0.8731
Epoch 744/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.4911 - accuracy: 0.8449 - val_loss: 0.4703 - val_accuracy: 0.8694
Epoch 745/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.4908 - accuracy: 0.8458 - val_loss: 0.4697 - val_accuracy: 0.8694
Epoch 746/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.4907 - accuracy: 0.8467 - val_loss: 0.4696 - val_accuracy: 0.8694
Epoch 747/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.4909 - accuracy: 0.8458 - val_loss: 0.4698 - val_accuracy: 0.8694
Epoch 748/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.4910 - accuracy: 0.8467 - val_loss: 0.4700 - val_accuracy: 0.8694
Epoch 749/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.4911 - accuracy: 0.8449 - val_loss: 0.4699 - val_accuracy: 0.8694
Epoch 750/1000

2/2 [==============================] - 0s 16ms/step - loss: 0.4827 - accuracy: 0.8458 - val_loss: 0.4661 - val_accuracy: 0.8731
Epoch 858/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.4830 - accuracy: 0.8458 - val_loss: 0.4659 - val_accuracy: 0.8731
Epoch 859/1000
2/2 [==============================] - 0s 17ms/step - loss: 0.4831 - accuracy: 0.8458 - val_loss: 0.4658 - val_accuracy: 0.8731
Epoch 860/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.4830 - accuracy: 0.8467 - val_loss: 0.4657 - val_accuracy: 0.8694
Epoch 861/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.4831 - accuracy: 0.8439 - val_loss: 0.4657 - val_accuracy: 0.8694
Epoch 862/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.4830 - accuracy: 0.8430 - val_loss: 0.4658 - val_accuracy: 0.8694
Epoch 863/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.4827 - accuracy: 0.8439 - val_loss: 0.4657 - val_accuracy: 0.8694
Epoch 864/1000

2/2 [==============================] - 0s 15ms/step - loss: 0.4765 - accuracy: 0.8439 - val_loss: 0.4705 - val_accuracy: 0.8619
Epoch 972/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.4769 - accuracy: 0.8430 - val_loss: 0.4705 - val_accuracy: 0.8619
Epoch 973/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.4772 - accuracy: 0.8439 - val_loss: 0.4704 - val_accuracy: 0.8619
Epoch 974/1000
2/2 [==============================] - 0s 16ms/step - loss: 0.4774 - accuracy: 0.8439 - val_loss: 0.4702 - val_accuracy: 0.8619
Epoch 975/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.4773 - accuracy: 0.8430 - val_loss: 0.4704 - val_accuracy: 0.8694
Epoch 976/1000
2/2 [==============================] - 0s 14ms/step - loss: 0.4773 - accuracy: 0.8449 - val_loss: 0.4705 - val_accuracy: 0.8694
Epoch 977/1000
2/2 [==============================] - 0s 15ms/step - loss: 0.4773 - accuracy: 0.8467 - val_loss: 0.4702 - val_accuracy: 0.8657
Epoch 978/1000

In [18]:
eval_results = model.evaluate(X_test, y_test)
eval_results

9/9 [==============================] - 0s 873us/step - loss: 0.4651 - accuracy: 0.8731


[0.4651351571083069, 0.8731343150138855]